### Basic Housekeeping

In [ ]:
## Import General Packages

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re


## Import Selenium Specific Functions

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.service import Service as SafariService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Compile the Quarterback Stats Dataset

In [ ]:
## Initialize Variables

names1 = []
names = []
teams = []
games = []
years = []
years1 = []
r_atts = []
r_yds = []
ypc = []
r_tds = []
qbs = []
qb_yards = []
qb_att = []
qb_ypa = []
qb_cmp = []
qb_td = []
qb_int = []
qb_rate = []
qb_sack = []

df = pd.DataFrame()

i = 0
n_years = 5


## Create function to make sure data is collected for the correct season
 
def yearMatch(season, soup, i = 0):
    year = soup[i].find('td').text.strip()
    if year == season:
        return soup[i].find_all('td')
    else:
        return(yearMatch(season, soup, i + 1))


## Main Web Scraping Loop

while i < n_years:
    try:
        driver.quit()
    except:
        print("No Driver")

    link = 'https://www.nfl.com/stats/player-stats/category/passing/' +str(2024 - i) +'/reg/all/passingyards/desc'
    driver = webdriver.Safari(service=SafariService())
    driver.get(link)
    while True:
        try:
            container = driver.find_element(By.CLASS_NAME, 'nfl-c-player-directory')
            all_qbs = container.find_elements(By.XPATH, ".//tbody/tr")
        except:
            break

        for player in all_qbs:
            stats = player.find_elements(By.XPATH, "./td")
            #years1.append(2024-i)
            qbs.append(stats[0].text.strip())
            names1.append(stats[0].text.strip())
            qb_yards.append(stats[1].text.strip())
            qb_att.append(stats[3].text.strip())
            qb_ypa.append(stats[2].text.strip())
            qb_cmp.append(stats[4].text.strip())
            qb_td.append(stats[6].text.strip())
            qb_int.append(stats[7].text.strip())
            qb_rate.append(stats[8].text.strip())
            qb_sack.append(stats[14].text.strip())

        try:
            next_button = container.find_element(By.XPATH, './/a[@class="nfl-o-table-pagination__next"]')
            next_button.click()
            time.sleep(2)

        except:
            break

    for qb in qbs:
        qb_name = re.sub('[^a-zA-Z]+','-', qb)
        url = 'https://www.nfl.com/players/' + qb_name + '/stats/career'
        #print(url.replace('-/','/'))
        r = requests.get(url.replace('-/','/'))
        bs = BeautifulSoup(r.text)
        try:
            rush = bs.find_all('div', {'class':'nfl-o-roster'})[1].find('tbody').find_all('tr')
            stats = yearMatch(str(2024-i), rush)
        except:
            continue
        names.append(qb)
        years.append(stats[0].text.strip())
        teams.append(stats[1].text.strip())
        games.append(stats[2].text.strip())
        r_atts.append(stats[3].text.strip())
        r_yds.append(stats[4].text.strip())
        ypc.append(stats[5].text.strip())
        r_tds.append(stats[7].text.strip())

    i += 1

    pass_df = pd.DataFrame({'QB':names1, 'YDS':qb_yards, 'ATT':qb_att, 'YPA':qb_ypa, 'CMP':qb_cmp, 'TDs':qb_td, 'INTs':qb_int, 'QBR':qb_rate, 'SCK':qb_sack})
    rush_df = pd.DataFrame({'QB':names, 'Team':teams, 'GP':games, 'Year':years, 'ATT(R)':r_atts, 'YDS(R)':r_yds, 'YPC':ypc, 'TDs(R)':r_tds})
    qb_df = pd.merge(rush_df, pass_df, on='QB')
    df = pd.concat([df, qb_df], ignore_index=True)

    names1 = []
    names = []
    teams = []
    games = []
    years = []
    r_atts = []
    r_yds = []
    ypc = []
    r_tds = []
    qbs = []
    qb_yards = []
    qb_att = []
    qb_ypa = []
    qb_cmp = []
    qb_td = []
    qb_int = []
    qb_rate = []
    qb_sack = []

driver.quit()
df = df.drop_duplicates(ignore_index=True)
df.to_csv('./qb_data.csv')


### Compile the Wide Reciever Stats Dataset

In [ ]:
## Initialize Variables

Recievers = []
Recs = []
Rec_Yds = []
Rec_TDs = []
Rec_YAC = []
Targets = []
Names = []
Years = []
Teams = []
Games = []

df = pd.DataFrame()

i = 0
n_years = 4


## Create function to make sure data is collected for the correct season 
## (Technically not needed as the function is included above, but including
##  it here saves me from having to get the function in my environment)

def yearMatch(season, soup, i = 0):
    year = soup[i].find('td').text.strip()
    if year == season:
        return(soup[i].find_all('td'))
    else:
        return(yearMatch(season, soup, i + 1))


## Main Web Scraping Loop

while i < n_years:
    try:
        driver.quit()
    except:
        print(f"No Driver")

    link = 'https://www.nfl.com/stats/player-stats/category/receiving/' +str(2024 - i) +'/reg/all/receivingreceptions/desc'
    driver = webdriver.Safari(service=SafariService())
    driver.get(link)
    while True:
        try:
            container = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'nfl-c-player-directory')))
            all_recievers = container.find_elements(By.XPATH, ".//tbody/tr")
        except:
            break
        
        for player in all_recievers:
            stats = player.find_elements(By.XPATH, "./td")
            Recievers.append(stats[0].text.strip())
            Recs.append(stats[1].text.strip())
            Rec_TDs.append(stats[3].text.strip())
            Rec_Yds.append(stats[2].text.strip())
            Rec_YAC.append(stats[10].text.strip())
            Targets.append(stats[11].text.strip())

        try:
            next_button = container.find_element(By.XPATH, './/a[@class="nfl-o-table-pagination__next"]')
            next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(all_recievers[0]))

        except:
            break

    for reciever in Recievers:
        name = re.sub('[^a-zA-Z]+','-', reciever)
        url = 'https://www.nfl.com/players/' + name + '/stats/career'
        #print(url.replace('-/','/'))
        r = requests.get(url.replace('-/','/'))
        bs = BeautifulSoup(r.text)
        try:
            rec = bs.find_all('div', {'class':'nfl-o-roster'})[0].find('tbody').find_all('tr')
            stats = yearMatch(str(2024-i), rec)
        except:
            Names.append(reciever)
            Years.append('NaN')
            Teams.append('NaN')
            Games.append('NaN')
            continue
        Names.append(reciever)
        Years.append(stats[0].text.strip())
        Teams.append(stats[1].text.strip())
        Games.append(stats[2].text.strip())

    i += 1
    rec_info = pd.DataFrame({'Reciever':Names, 'Team':Teams, 'Games':Games, 'Year':Years})
    rec_df = pd.DataFrame({'Reciever':Recievers, 'Receptions':Recs, 'REC_YDS':Rec_Yds, 'REC_TDS':Rec_TDs, 'REC_YAC':Rec_YAC, 'Targets':Targets})
    full_rec_df = pd.merge(rec_info, rec_df, on='Reciever')
    df = pd.concat([df, full_rec_df], ignore_index=True)

    Recievers = []
    Recs = []
    Rec_Yds = []
    Rec_TDs = []
    Rec_YAC = []
    Targets = []
    Names = []
    Years = []
    Teams = []
    Games = []
    
driver.quit()
df = df.drop_duplicates(ignore_index=True)
df.to_csv('./wr_full.csv')
